In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

#Load and process the data
(X_train,_),(X_test,_) = mnist.load_data()
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
print(X_train.shape)
print(X_test.shape)

X_train = X_train.reshape(len(X_train), np.prod(X_train.shape[1: ]))
X_test = X_test.reshape(len(X_test), np.prod(X_test.shape[1: ]))
print(X_train.shape)
print(X_test.shape)

# **Simple Autoencoder**

#Define encoder architecture
input_img = Input(shape=(784, ))
encoded = Dense(128, activation = 'sigmoid')(input_img)
decoded = Dense(784, activation = 'sigmoid')(encoded)

#Build an encoder model
autoencoder = Model(input_img, decoded)

#Compile the model
autoencoder.compile(optimizer = Adam(), loss='binary_crossentropy')

#Train the model
autoencoder.fit(X_train, X_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data = (X_test, X_test))

import matplotlib.pyplot as plt

#Encode and Decode
encode_imgs = autoencoder.predict(X_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(encode_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

# **Denoising Autoencoder**

# Add noise to the input data
noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)
print(X_train_noisy.shape)
print(X_test_noisy.shape)

X_train = X_train.reshape(len(X_train),np.prod(X_train.shape[1:]))
X_test = X_test.reshape(len(X_test),np.prod(X_test.shape[1:]))
X_train_noisy = X_train_noisy.reshape(len(X_train_noisy), np.prod(X_train_noisy.shape[1:]))
X_test_noisy = X_test_noisy.reshape(len(X_test_noisy), np.prod(X_test_noisy.shape[1:]))
print(X_train.shape)
print(X_test.shape)

#Train the model
autoencoder.fit(X_train_noisy,X_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test_noisy,X_test))

decoded_imgs = autoencoder.predict(X_test_noisy)

# Display the original, noisy, and reconstructed images
n = 8
plt.figure(figsize=(20, 6))
for i in range(n):
    # Display original images
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display noisy images
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(X_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstructed images
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# **Sparse Autoencoder**

#Define sparse autoencoder
input_img1 = Input(shape=(784, ))
encodedSparse = Dense(128, activation = 'sigmoid', activity_regularizer = l1(10e-5))(input_img1)
decodedSparse = Dense(784, activation = 'sigmoid')(encodedSparse)

#Build an encoder model
autoencoderSparse = Model(input_img1, decodedSparse)

#Compile the model
autoencoderSparse.compile(optimizer = Adam(), loss='binary_crossentropy')

#Train the model
autoencoderSparse.fit(X_train, X_train,
                epochs=7,
                batch_size=256,
                shuffle=True,
                validation_data = (X_test, X_test))

#Encode and Decode
encode_imgs1 = autoencoderSparse.predict(X_test)

n = 8
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(encode_imgs1[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()